<a href="https://colab.research.google.com/github/ffserro/MVP/blob/master/mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ffserro/MVP/blob/master/mvp.ipynb)


In [35]:
!git clone 'https://github.com/ffserro/MVP.git'
!pip install -r '/content/MVP/requirements.txt' > '/content/pip_log.txt'
!if grep -iq "restart" '/content/pip_log.txt';then python -c "import os;os.kill(os.getpid(), 9)"; else python -c "print('Vamos começar!')"; fi

fatal: destination path 'MVP' already exists and is not an empty directory.
Vamos começar!


In [26]:
!grep 'restart' '/content/test.txt'

In [31]:
!python -c "print('hello');print('again')"

hello
again


In [34]:
!if grep -iq "restart" '/content/test.txt';then python -c "print('its there')"; else python -c "print('its not')"; fi

its not


In [18]:
from glob import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


import pmdarima as pm
from prophet import Prophet
from xgboost import XGBRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [19]:
mmm = pd.DataFrame()
mmm = pd.concat([mmm]+[pd.read_excel(d) for d in glob('/content/MVP/dados/mmm/*')]).reset_index(drop=True)

etapas = pd.DataFrame()
etapas = pd.concat([etapas]+[pd.read_excel(d) for d in glob('/content/MVP/dados/etapas/*')]).reset_index(drop=True).rename(columns={'uasg':'codigo'})

da = pd.DataFrame()
da = pd.concat([da]+[pd.read_excel(d) for d in glob('/content/MVP/dados/da/*')]).reset_index(drop=True)

In [20]:
mmm.codigo.value_counts().describe()

,count
count,247.000000
mean,50.842105
std,32.580035
min,1.000000
25%,13.500000
50%,70.000000
75%,80.000000
max,80.000000


In [21]:
da.groupby(['mes', 'ano']).codigo.count().describe()

,codigo
count,80.000000
mean,357.900000
std,38.667554
min,102.000000
25%,334.750000
50%,369.500000
75%,386.000000
max,404.000000


In [22]:
etapas[['mes', 'ano', 'uasg']].drop_duplicates().groupby(['mes', 'ano']).uasg.count().describe()

KeyError: "['uasg'] not in index"

In [ ]:
mmm_marinha = mmm.groupby(['ano', 'mes'])[[col for col in mmm.columns if col not in ['ano', 'mes', 'codigo', 'nome']]].sum().reset_index()

In [ ]:
mmm_marinha['mes'] = [''.join([str(i[0]), '_', str(i[1])]) for i in zip(mmm_marinha.mes.values, mmm_marinha.ano.values)]
mmm_marinha.drop(columns=['ano'], inplace=True)
mmm_marinha = mmm_marinha.iloc[:-2]

In [ ]:
mmm

In [ ]:
mmm_receita_despesa = pd.merge(mmm, da, how='inner', on=['mes', 'ano', 'codigo'])

In [ ]:
mmm_receita_despesa[abs((mmm_receita_despesa.despesa_autorizada_global - (mmm_receita_despesa.generos_consumidos + mmm_receita_despesa.vale_extra)) - mmm_receita_despesa.sobra_licita) < 1]

In [ ]:
mmm_receita_despesa[mmm_receita_despesa.codigo==80000]

In [ ]:
mmm_receita_despesa.codigo.value_counts()

In [ ]:
etapas[etapas.codigo_etapa.isin([102, 104])]

In [ ]:
mmm_etapas = pd.merge(left=mmm, right=etapas, how='inner', on=['ano', 'mes', 'codigo'])

In [ ]:
mmm_etapas[mmm_etapas.codigo_etapa.isin([103, 105])][['ano', 'mes', 'nome', 'codigo_etapa', 'quantidade']]

In [ ]:
pd.DataFrame({
    'diferenca': mmm_receita_despesa.despesa_autorizada_global - (mmm_receita_despesa.generos_consumidos + mmm_receita_despesa.vale_extra),
    'sobra_licita': mmm_receita_despesa.sobra_licita
})

In [ ]:
da_marinha = da.groupby(['mes', 'ano']).despesa_autorizada_global.sum().reset_index()
da_marinha['mes'] = ['_'.join(['{:02d}'.format(i[0]), str(i[1])]) for i in zip(da_marinha.mes.values, da_marinha.ano.values)]
da_marinha = da_marinha.sort_values(by=['ano', 'mes']).iloc[:-2]
fig = px.line(
    da_marinha,
    x = 'mes',
    y = 'despesa_autorizada_global',
    title='Despesa Autorizada global'
)

fig.show()

In [ ]:
def grafico_base(titulo):
    return px.line(
        mmm_marinha,
        x = 'mes',
        y = 'totais_balanco_paiol_despesa',
        labels = {
            'mes': 'Mês e ano',
            'totais_balanco_paiol_despesa': 'Totais das despesas'
        },
        title = titulo
    )

fig = grafico_base('Gastos com alimentação dos últimos cinco anos')

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Teste de estacionariedade

result = adfuller(mmm_marinha.totais_balanco_paiol_despesa)
print(f'ADF: {result[0]}, p-valor: {result[1]}')

In [ ]:
naive_forecast = mmm_marinha.totais_balanco_paiol_despesa.shift(1)
mae_naive = mean_absolute_error(mmm_marinha.totais_balanco_paiol_despesa.iloc[1:], naive_forecast.iloc[1:])
print('Baseline Naïve MAE:', mae_naive)

In [ ]:
# SARIMA

train = mmm_marinha.iloc[:-12, :]
test = mmm_marinha.iloc[-12:, :]

model_auto = pm.auto_arima(train.totais_balanco_paiol_despesa, seasonal=True, m=12, stepwise=True, trace=True)
print(model_auto.summary())

In [ ]:
mmm

In [ ]:
import ipywidgets as widgets
from IPython.display import display

w = widgets.Dropdown(
    options=['Addition', 'Multiplication', 'Subtraction', 'Division'],
    value='Addition',
    description='Task:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

w.observe(on_change)

display(w)

In [ ]:
color = 'green' #@param ["red", "green", "blue"]
print(f"Selected color: {color}")

In [ ]:
sarima_model = SARIMAX(train.totais_balanco_paiol_despesa,
                       order=model_auto.order,
                       seasonal_order=model_auto.seasonal_order)

sarima_fit = sarima_model.fit(disp=False)

forecast_sarima = sarima_fit.get_forecast(steps=12)
pred_sarima = forecast_sarima.predicted_mean

mae_sarima = mean_absolute_error(test.totais_balanco_paiol_despesa, pred_sarima)
print('SARIMA MAE:', mae_sarima)



In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo SARIMA')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_sarima
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Previsão com o Prophet

prophet_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
prophet_df.columns = ['ds', 'y']
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'], format='%m_%Y')

model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.fit(prophet_df)

future = model_prophet.make_future_dataframe(periods=12, freq='M')
forecast = model_prophet.predict(future)

forecast_test = forecast.set_index('ds').loc[pd.to_datetime(test.mes, format='%m_%Y')]
mae_prophet = mean_absolute_error(test['totais_balanco_paiol_despesa'], forecast_test['yhat'])
print('Prophet MAE', mae_prophet)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo Prophet')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = forecast_test.yhat
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Regressão com XGBoost para séries temporais

xg_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
xg_df['ano'] = xg_df['mes'].apply(lambda x: x.split('_')[-1]).astype(int)
xg_df['mes'] = xg_df['mes'].apply(lambda x: x.split('_')[0]).astype(int)
xg_df['lag1'] = xg_df['totais_balanco_paiol_despesa'].shift(1)
xg_df['lag3'] = xg_df['totais_balanco_paiol_despesa'].shift(3)
xg_df['lag6'] = xg_df['totais_balanco_paiol_despesa'].shift(6)
xg_df['rolling3'] = xg_df['totais_balanco_paiol_despesa'].rolling(3).mean()
xg_df['rolling6'] = xg_df['totais_balanco_paiol_despesa'].rolling(6).mean()

xg_df = xg_df.dropna().reset_index(drop=True)

In [ ]:
train_xg = xg_df.iloc[:-12]
test_xg = xg_df.iloc[-12:]

X_train = train_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_train = train_xg['totais_balanco_paiol_despesa']
X_test = test_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_test = test_xg['totais_balanco_paiol_despesa']

xgb = XGBRegressor(
    n_estimators = 300,
    learning_rate = 0.05,
    max_depth = 5,
    subsample = 0.8,
    colsample_bytree = 0.8,
    random_state = 42
)

xgb.fit(X_train, y_train)

pred_xgb = xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, pred_xgb)

print("XGBoost MAE:", mae_xgb)


In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo XGBoost')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_xgb
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
hw_model = ExponentialSmoothing(
    train['totais_balanco_paiol_despesa'],
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit()

pred_hw = hw_model.forecast(12)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo ExponentialSmoothing')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_hw
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
despesas_scaled = scaler.fit_transform(mmm_marinha.totais_balanco_paiol_despesa.values.reshape(-1, 1))

def create_sequences(data, window=12):
    X, y = [], []
    for i in range(len(data)- window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(despesas_scaled)


split = len(X) - 12
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('Shape treino:', X_train.shape, y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='tanh', return_sequences=True, input_shape=(12, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae')

history = model.fit(
    X_train, y_train,
    epochs = 200,
    batch_size = 4,
    validation_split = 0.1,
    verbose = 1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm = mean_absolute_error(y_test_inv, y_pred_inv)
print('LSTM MAE:', mae_lstm)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.reshape(1, -1)[0]
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
def create_sequences_multistep(data, window=12, horizon=12):
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(data[i+window:i+window+horizon].flatten())
    return np.array(X), np.array(y)

window = 12
horizon = 12
X, y = create_sequences_multistep(despesas_scaled, window, horizon)

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('X_train shape:', X_train.shape)
print('y_train, shape:', y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation="tanh", return_sequences=True, input_shape=(window, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation="tanh"))
model.add(Dense(horizon))  # saída com 12 valores (multi-step)

model.compile(optimizer="adam", loss="mse")

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=4,
    validation_split=0.1,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm_multi = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
print('LSTM Multi-step MAE:', mae_lstm_multi)

In [ ]:
last_input_dates = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-(window + horizon):-horizon]
future_dates = pd.date_range(start = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-horizon], periods=horizon, freq='M')

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM Multi-step')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.flatten()
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
df['mes/ano'] = ['/'.join([str(i[0]), str(i[1])]) for i in zip(df.mes, df.ano)]

plt.figure(figsize=(100, 20))
px.line(df, x='mes/ano', y='saldo', color='nome')